In [1]:
import pandas as pd
import pathlib
import pickle
import time
import requests

In [2]:
data = pd.read_csv('/home/bh_parijat/data/main-dataset.csv')

In [3]:
not_present = data[data['zipcode'].isnull()]

In [4]:
not_present.shape

(464464, 21)

In [39]:
not_present.head(5000)

,index,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,...,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,org_index,zipcode
180000,180000,164571494,05/08/2017,339.0,"LARCENY,PETIT FROM OPEN AREAS,UNCLASSIFIED",341.0,PETIT LARCENY,PL 1552500,M,K,...,0.0,45-64,M,WHITE HISPANIC,981144.0,174000.0,40.644274,-74.011194,NaN,NaN
180001,180001,164558531,05/08/2017,439.0,"LARCENY,GRAND FROM OPEN AREAS,UNCLASSIFIED",109.0,GRAND LARCENY,PL 1553004,F,M,...,0.0,45-64,M,WHITE,984984.0,200116.0,40.715957,-73.997354,NaN,NaN
180002,180002,164571491,05/08/2017,779.0,"PUBLIC ADMINISTRATION,UNCLASSIFIED FELONY",126.0,MISCELLANEOUS PENAL LAW,PL 2155100,F,K,...,0.0,25-44,M,WHITE,996773.0,156390.0,40.595930,-73.954908,NaN,NaN
180003,180003,164570089,05/08/2017,494.0,"STOLEN PROPERTY 2,1,POSSESSION,UNCLASSIFIED",111.0,POSSESSION OF STOLEN PROPERTY,PL 1654501,F,M,...,0.0,45-64,M,WHITE HISPANIC,988049.0,200071.0,40.715833,-73.986297,NaN,NaN
180004,180004,164558610,05/08/2017,779.0,"PUBLIC ADMINISTRATION,UNCLASSIFIED FELONY",126.0,MISCELLANEOUS PENAL LAW,PL 215510B,F,Q,...,0.0,25-44,M,ASIAN / PACIFIC ISLANDER,1057767.0,203993.0,40.726293,-73.734761,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184995,184995,164376279,05/02/2017,104.0,VEHICULAR ASSAULT (INTOX DRIVE,126.0,MISCELLANEOUS PENAL LAW,PL 1200301,F,M,...,0.0,45-64,M,ASIAN / PACIFIC ISLANDER,986858.0,211975.0,40.748507,-73.990589,NaN,NaN
184996,184996,164334223,05/02/2017,922.0,"TRAFFIC,UNCLASSIFIED MISDEMEANOR",348.0,VEHICLE AND TRAFFIC LAWS,VTL0511002,M,B,...,0.0,25-44,M,WHITE HISPANIC,1025011.0,241786.0,40.830236,-73.852711,NaN,NaN
184997,184997,164333035,05/02/2017,567.0,"MARIJUANA, POSSESSION 4 & 5",235.0,DANGEROUS DRUGS,PL 2211001,M,M,...,0.0,18-24,F,ASIAN / PACIFIC ISLANDER,1000098.0,249298.0,40.850934,-73.942717,NaN,NaN
184998,184998,164377282,05/02/2017,511.0,"CONTROLLED SUBSTANCE, POSSESSION 7",235.0,DANGEROUS DRUGS,PL 2200300,M,K,...,0.0,18-24,M,WHITE,976087.0,163271.0,40.614822,-74.029403,NaN,NaN


In [7]:
lat_long = not_present[['Longitude','Latitude']]

In [59]:
def reverse_geocode(base_url,query_params,k = 1,num_cpus=1):
    
    
    path = pathlib.Path().absolute()/'reverse-data-map'/'not_present'
    
    start_time = time.time()
    chunk = lat_long.shape[0]//num_cpus
    
    start = None
    
    end = None
    
    prev = start
    print(chunk,start,end)
    indices, zipcodes = [],[]
    index,zipcode = [],[]
    
    latitude,longitude = [],[]
    
    latitudes,longitudes = [],[]
    
    counter = 0
    for i,row in lat_long.iloc[:,:].iterrows():
        counter+=1
        
        if start is None:
            start = i
            prev = start
            
            
        url = base_url + "point.lon=" + str(row[0]) + "&point.lat=" + str(row[1]) + query_params
        res = requests.request('GET',url).json()
        #print(url)
        if counter%20000 == 0:
            taken = (time.time()-start_time)
            percent_done = (counter)*100/lat_long.shape[0]
            print("{} done after {} on cpu {}".format(percent_done,taken,k))
            
            name= str(prev)+"result"+str(i)+".pkl"
            
            where_save = path / name
            prev = i
            
            with open(where_save, 'wb') as handle:
                pickle.dump(zip(index,zipcode,latitude,longitude), handle, protocol=pickle.HIGHEST_PROTOCOL)
            
            index = []
            zipcode = []
            latitude = []
            longitude = []
            
        features = res['features']
        
        postal_code = None
        
        for feature in features:
            if 'postalcode' in feature['properties']:
                postal_code = feature['properties']['postalcode']
                break
        
        
    
        if postal_code is None:
            #not_present[i] = 1
            print("postcode not present for i = {} on cpu {}".format(i,k))
            continue

       
        
        index.append(i)
        zipcode.append(postal_code)
        indices.append(i)
        zipcodes.append(postal_code)
        
        
        latitudes.append(row[1])
        latitude.append(row[1])
        
        
        longitudes.append(row[0])
        longitude.append(row[0])
        
        
    
    print(" cpu {} done after {}".format(k,taken))
    return indices,postal_code,latitudes,longitudes

In [60]:
base_url = "http://34.83.157.143:4000/v1/reverse?"
query_params = "&size=10"
indices, postal_code,latitude,longitudes = reverse_geocode(base_url=base_url,query_params=query_params)

464464 None None
postcode not present for i = 183269 on cpu 1
postcode not present for i = 187943 on cpu 1
postcode not present for i = 190835 on cpu 1
postcode not present for i = 191365 on cpu 1
postcode not present for i = 195217 on cpu 1
postcode not present for i = 196976 on cpu 1
postcode not present for i = 197332 on cpu 1
postcode not present for i = 197620 on cpu 1
postcode not present for i = 197895 on cpu 1
4.30603878879741 done after 418.5952408313751 on cpu 1
postcode not present for i = 380454 on cpu 1
postcode not present for i = 380716 on cpu 1
postcode not present for i = 384027 on cpu 1
postcode not present for i = 384469 on cpu 1
postcode not present for i = 384661 on cpu 1
postcode not present for i = 385456 on cpu 1
postcode not present for i = 386276 on cpu 1
postcode not present for i = 386372 on cpu 1
postcode not present for i = 386821 on cpu 1
postcode not present for i = 389940 on cpu 1
postcode not present for i = 391075 on cpu 1
postcode not present for i =

postcode not present for i = 1593703 on cpu 1
postcode not present for i = 1594095 on cpu 1
postcode not present for i = 1594421 on cpu 1
postcode not present for i = 1594822 on cpu 1
postcode not present for i = 1595415 on cpu 1
postcode not present for i = 1597873 on cpu 1
postcode not present for i = 1597986 on cpu 1
postcode not present for i = 1598680 on cpu 1
postcode not present for i = 1598895 on cpu 1
postcode not present for i = 1598897 on cpu 1
postcode not present for i = 1781218 on cpu 1
postcode not present for i = 1781226 on cpu 1
postcode not present for i = 1781874 on cpu 1
postcode not present for i = 1781898 on cpu 1
34.44831031037928 done after 3380.6913657188416 on cpu 1
postcode not present for i = 1783006 on cpu 1
postcode not present for i = 1783008 on cpu 1
postcode not present for i = 1783269 on cpu 1
postcode not present for i = 1783400 on cpu 1
postcode not present for i = 1784914 on cpu 1
postcode not present for i = 1786359 on cpu 1
postcode not present fo

postcode not present for i = 1926575 on cpu 1
postcode not present for i = 1927073 on cpu 1
postcode not present for i = 1927247 on cpu 1
postcode not present for i = 1927257 on cpu 1
postcode not present for i = 1928088 on cpu 1
postcode not present for i = 1928219 on cpu 1
postcode not present for i = 1928227 on cpu 1
postcode not present for i = 1929482 on cpu 1
postcode not present for i = 1930466 on cpu 1
postcode not present for i = 1931473 on cpu 1
postcode not present for i = 1932096 on cpu 1
postcode not present for i = 1932234 on cpu 1
postcode not present for i = 1932266 on cpu 1
postcode not present for i = 1933003 on cpu 1
postcode not present for i = 1933353 on cpu 1
postcode not present for i = 1933440 on cpu 1
postcode not present for i = 1934324 on cpu 1
postcode not present for i = 1934404 on cpu 1
postcode not present for i = 1935323 on cpu 1
postcode not present for i = 1935440 on cpu 1
postcode not present for i = 1935728 on cpu 1
postcode not present for i = 19357

postcode not present for i = 1987812 on cpu 1
postcode not present for i = 1987885 on cpu 1
postcode not present for i = 1988545 on cpu 1
postcode not present for i = 1988944 on cpu 1
postcode not present for i = 1988990 on cpu 1
postcode not present for i = 1989172 on cpu 1
postcode not present for i = 1989262 on cpu 1
postcode not present for i = 1989265 on cpu 1
postcode not present for i = 1989581 on cpu 1
postcode not present for i = 1989635 on cpu 1
postcode not present for i = 1990147 on cpu 1
postcode not present for i = 1990635 on cpu 1
postcode not present for i = 1991085 on cpu 1
postcode not present for i = 1991142 on cpu 1
postcode not present for i = 1991525 on cpu 1
postcode not present for i = 1991688 on cpu 1
postcode not present for i = 1991846 on cpu 1
postcode not present for i = 1991893 on cpu 1
postcode not present for i = 1992242 on cpu 1
postcode not present for i = 1992454 on cpu 1
postcode not present for i = 1992463 on cpu 1
postcode not present for i = 19925

postcode not present for i = 2045932 on cpu 1
postcode not present for i = 2047231 on cpu 1
postcode not present for i = 2047241 on cpu 1
postcode not present for i = 2047616 on cpu 1
postcode not present for i = 2047821 on cpu 1
postcode not present for i = 2047901 on cpu 1
postcode not present for i = 2048013 on cpu 1
postcode not present for i = 2048671 on cpu 1
postcode not present for i = 2049345 on cpu 1
postcode not present for i = 2049470 on cpu 1
postcode not present for i = 2049856 on cpu 1
postcode not present for i = 2050119 on cpu 1
postcode not present for i = 2050454 on cpu 1
postcode not present for i = 2050487 on cpu 1
postcode not present for i = 2050813 on cpu 1
postcode not present for i = 2051025 on cpu 1
postcode not present for i = 2051070 on cpu 1
postcode not present for i = 2051226 on cpu 1
postcode not present for i = 2051621 on cpu 1
postcode not present for i = 2051654 on cpu 1
postcode not present for i = 2051655 on cpu 1
postcode not present for i = 20521

...

postcode not present for i = 3079277 on cpu 1
postcode not present for i = 3079327 on cpu 1
postcode not present for i = 3079492 on cpu 1
postcode not present for i = 3079622 on cpu 1
postcode not present for i = 3079696 on cpu 1
postcode not present for i = 3079999 on cpu 1
postcode not present for i = 3080089 on cpu 1
postcode not present for i = 3080747 on cpu 1
postcode not present for i = 3081366 on cpu 1
postcode not present for i = 3081437 on cpu 1
postcode not present for i = 3081641 on cpu 1
postcode not present for i = 3081662 on cpu 1
postcode not present for i = 3082120 on cpu 1
postcode not present for i = 3082552 on cpu 1
postcode not present for i = 3082624 on cpu 1
postcode not present for i = 3083126 on cpu 1
postcode not present for i = 3083932 on cpu 1
postcode not present for i = 3085088 on cpu 1
postcode not present for i = 3085156 on cpu 1
postcode not present for i = 3085377 on cpu 1
postcode not present for i = 3085461 on cpu 1
postcode not present for i = 30854

postcode not present for i = 3139238 on cpu 1
postcode not present for i = 3139488 on cpu 1
postcode not present for i = 3139647 on cpu 1
postcode not present for i = 3140608 on cpu 1
postcode not present for i = 3140869 on cpu 1
postcode not present for i = 3141414 on cpu 1
postcode not present for i = 3141680 on cpu 1
postcode not present for i = 3141842 on cpu 1
postcode not present for i = 3143863 on cpu 1
postcode not present for i = 3143912 on cpu 1
postcode not present for i = 3143991 on cpu 1
postcode not present for i = 3144305 on cpu 1
postcode not present for i = 3144603 on cpu 1
postcode not present for i = 3145388 on cpu 1
postcode not present for i = 3146144 on cpu 1
postcode not present for i = 3146489 on cpu 1
postcode not present for i = 3146890 on cpu 1
postcode not present for i = 3148615 on cpu 1
postcode not present for i = 3149096 on cpu 1
postcode not present for i = 3149757 on cpu 1
postcode not present for i = 3149958 on cpu 1
postcode not present for i = 31500

postcode not present for i = 3237783 on cpu 1
postcode not present for i = 3237945 on cpu 1
postcode not present for i = 3237988 on cpu 1
postcode not present for i = 3238679 on cpu 1
postcode not present for i = 3239492 on cpu 1
postcode not present for i = 3239567 on cpu 1
postcode not present for i = 3240882 on cpu 1
postcode not present for i = 3242109 on cpu 1
postcode not present for i = 3243331 on cpu 1
postcode not present for i = 3243615 on cpu 1
postcode not present for i = 3244333 on cpu 1
postcode not present for i = 3244339 on cpu 1
postcode not present for i = 3244416 on cpu 1
postcode not present for i = 3245586 on cpu 1
postcode not present for i = 3246184 on cpu 1
postcode not present for i = 3246794 on cpu 1
postcode not present for i = 3247041 on cpu 1
postcode not present for i = 3247338 on cpu 1
postcode not present for i = 3247405 on cpu 1
postcode not present for i = 3247809 on cpu 1
postcode not present for i = 3248172 on cpu 1
postcode not present for i = 32483

postcode not present for i = 3443496 on cpu 1
postcode not present for i = 3445296 on cpu 1
postcode not present for i = 3449124 on cpu 1
postcode not present for i = 3450480 on cpu 1
postcode not present for i = 3452261 on cpu 1
postcode not present for i = 3453222 on cpu 1
postcode not present for i = 3458429 on cpu 1
postcode not present for i = 3459388 on cpu 1
postcode not present for i = 3462654 on cpu 1
postcode not present for i = 3465404 on cpu 1
postcode not present for i = 3471174 on cpu 1
postcode not present for i = 3471177 on cpu 1
postcode not present for i = 3471801 on cpu 1
postcode not present for i = 3473577 on cpu 1
postcode not present for i = 3475892 on cpu 1
postcode not present for i = 3478805 on cpu 1
postcode not present for i = 3481370 on cpu 1
postcode not present for i = 3481384 on cpu 1
postcode not present for i = 3481628 on cpu 1
postcode not present for i = 3483362 on cpu 1
postcode not present for i = 3484390 on cpu 1
postcode not present for i = 34914

postcode not present for i = 4381122 on cpu 1
postcode not present for i = 4385896 on cpu 1
90.4268145647456 done after 8774.242381095886 on cpu 1
postcode not present for i = 4394537 on cpu 1
postcode not present for i = 4398003 on cpu 1
postcode not present for i = 4400753 on cpu 1
postcode not present for i = 4438476 on cpu 1
postcode not present for i = 4450138 on cpu 1
postcode not present for i = 4469704 on cpu 1
postcode not present for i = 4519675 on cpu 1
postcode not present for i = 4544438 on cpu 1
postcode not present for i = 4582175 on cpu 1
postcode not present for i = 4582838 on cpu 1
postcode not present for i = 4584687 on cpu 1
postcode not present for i = 4584834 on cpu 1
postcode not present for i = 4585060 on cpu 1
postcode not present for i = 4585303 on cpu 1
postcode not present for i = 4587845 on cpu 1
postcode not present for i = 4588714 on cpu 1
postcode not present for i = 4590030 on cpu 1
postcode not present for i = 4591381 on cpu 1
94.732853353543 done afte

In [55]:
path

PosixPath('/home/bh_parijat/NYPD-crime-analysis/reverse-data-map/not-present')

In [63]:
df = pd.DataFrame(data={'org_indices':indices,'zipcode':postal_code,'Latitude':latitude,'Longitude':longitudes})

In [65]:
lat_long.shape[0]-df.shape[0]

6480

In [70]:
df.to_csv("/home/bh_parijat/NYPD-crime-analysis/reverse-data-map/not_present/remaining_points.csv",index=False)